<a href="https://colab.research.google.com/github/sam-thurman/asl_alphabet_image_classification/blob/master/notebooks/augment_exploratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import scipy as math
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
# from PIL import Image

from sklearn.model_selection import train_test_split
import keras

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

from keras.models import Model, Sequential
from keras import layers
from keras.layers import Dense, Input, Dropout, GlobalAveragePooling2D, Flatten, Conv2D, BatchNormalization, Activation, MaxPooling2D
from keras import optimizers
from keras.optimizers import Adam

import sys
import os


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
Using TensorFlow backend.


In [0]:
# mount google drive
from google.colab import drive
# drive.mount('/content/drive/My Drive/data')
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
%cd drive/My\ Drive/

/content/drive/My Drive


In [0]:
train_path = 'data/asl_alphabet_train'
validation_path = 'data/asl_alphabet_validation'
sam_path = 'data/asl_extra_val'

In [0]:
# image generator instance
datagen = keras.preprocessing.image.ImageDataGenerator(
                rescale=1./255,
                # width_shift_range=[-30,30],
                # height_shift_range=0.2,
                # rotation_range=90,
                # brightness_range=[0.2,.9],
                # zoom_range=[0.5,1.0],
                validation_split=0.3) 
valgen = keras.preprocessing.image.ImageDataGenerator(
                rescale=1./255) 
samgen = keras.preprocessing.image.ImageDataGenerator(
                rescale=1./255)

In [0]:

image_size = 128
batch_size = 32

print('done')
train_generator = datagen.flow_from_directory(
                train_path,  #directory for training images
                target_size=(image_size, image_size),
                batch_size=batch_size,
                class_mode='categorical',
                color_mode='grayscale',
                shuffle=True,
                subset='training')
print('done')
test_generator = datagen.flow_from_directory(
                train_path,  # same directory for testing images
                target_size=(image_size, image_size),
                batch_size=batch_size,
                class_mode='categorical',
                color_mode='grayscale',
                shuffle=True,
                subset='validation')
print('done')


done
Found 60991 images belonging to 29 classes.
done
Found 26134 images belonging to 29 classes.
done


In [0]:
val_generator = valgen.flow_from_directory(
                validation_path,  # directory for validation images
                target_size=(image_size, image_size),
                batch_size=batch_size,
                class_mode='categorical',
                color_mode='grayscale',
                shuffle=False)
print('done')
sam_generator = samgen.flow_from_directory(
                sam_path,  # directory for validation images
                target_size=(image_size, image_size),
                batch_size=batch_size,
                class_mode='categorical',
                color_mode='grayscale',
                shuffle=False)
print('done')

Found 30 images belonging to 29 classes.
done
Found 29 images belonging to 29 classes.
done


In [0]:
nb_classes = 29

# initialize CNN
model4 = Sequential()

# 1st conv layer
model4.add(Conv2D(32,(3,3), padding='same', input_shape=(image_size, image_size,1)))
model4.add(BatchNormalization())
model4.add(Activation('relu'))
model4.add(MaxPooling2D(pool_size=(2, 2)))
model4.add(Dropout(0.25))

# 2nd conv layer
model4.add(Conv2D(64,(5,5), padding='same'))
model4.add(BatchNormalization())
model4.add(Activation('relu'))
model4.add(MaxPooling2D(pool_size=(2, 2)))
model4.add(Dropout(0.25))

# 3rd conv layer
model4.add(Conv2D(128,(3,3), padding='same'))
model4.add(BatchNormalization())
model4.add(Activation('relu'))
model4.add(MaxPooling2D(pool_size=(2, 2)))
model4.add(Dropout(0.25))

# 4th conv layer
model4.add(Conv2D(128,(3,3), padding='same'))
model4.add(BatchNormalization())
model4.add(Activation('relu'))
model4.add(MaxPooling2D(pool_size=(2, 2)))
model4.add(Dropout(0.25))

# flattening
model4.add(Flatten())

# 1st fully connected layer
model4.add(Dense(128))
model4.add(BatchNormalization())
model4.add(Activation('relu'))
model4.add(Dropout(0.4))

# 2nd fully connected layer
model4.add(Dense(64))
model4.add(BatchNormalization())
model4.add(Activation('relu'))
model4.add(Dropout(0.3))

model4.add(Dense(nb_classes, activation='softmax'))



In [0]:
optimizer = Adam(lr=0.01)
model4.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
hisotry4 = model4.fit_generator(train_generator, validation_data=val_generator, epochs=5)

Epoch 1/5
1906/1906 [==============================] - 27400s 14s/step - loss: 3.3128 - accuracy: 0.0622 - val_loss: 2.8539 - val_accuracy: 0.1000
Epoch 2/5
1906/1906 [==============================] - 3893s 2s/step - loss: 3.1931 - accuracy: 0.0887 - val_loss: 2.6284 - val_accuracy: 0.3333
Epoch 3/5
1906/1906 [==============================] - 3911s 2s/step - loss: 3.0826 - accuracy: 0.1192 - val_loss: 2.5361 - val_accuracy: 0.5000
Epoch 4/5
1906/1906 [==============================] - 3905s 2s/step - loss: 2.9593 - accuracy: 0.1456 - val_loss: 1.9812 - val_accuracy: 0.4667
Epoch 5/5
1906/1906 [==============================] - 3889s 2s/step - loss: 2.8262 - accuracy: 0.1740 - val_loss: 2.1465 - val_accuracy: 0.2667


In [0]:
model4.evaluate_generator(sam_generator, verbose=1)

NameError: ignored

In [0]:
222/1906 [==>...........................] - ETA: 7:02:20 - loss: 3.4406 - accuracy: 0.0379